In [1]:
%pylab inline
from my_snn.tonic_dataloader import DatasetLoader
from my_snn.abstract_rsnn import CHECKPOINT_PATH

Populating the interactive namespace from numpy and matplotlib


In [2]:
dataset = 'shd'
time_window = 250
batch_size = 64 # lr=1e-4
#batch_size = 128 # lr=1e-4
DL = DatasetLoader(dataset=dataset, caching='memory', num_workers=0, batch_size=batch_size, time_window=time_window)
test_loader, train_loader = DL.get_dataloaders()

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR
import math
import torch.nn.functional as F
from torch.utils import data
import os

torch.manual_seed(2)

'''
STEP 2: MAKING DATASET ITERABLE
'''

decay = 0.1  # neuron decay rate
thresh = 0.5  # neuronal threshold
lens = 0.5  # hyper-parameters of approximate function
num_epochs = 20  # 150  # n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

'''
STEP 3a: CREATE spike MODEL CLASS
'''

b_j0 = 0.01  # neural threshold baseline
R_m = 1  # membrane resistance
dt = 1  #
gamma = .5  # gradient scale

# define approximate firing function

gradient_type = 'MG'
print('gradient_type: ',gradient_type)
scale = 6.
hight = 0.15
print('hight: ',hight,';scale: ',scale)

def gaussian(x, mu=0., sigma=.5):
    return torch.exp(-((x - mu) ** 2) / (2 * sigma ** 2)) / torch.sqrt(2 * torch.tensor(math.pi)) / sigma


# define approximate firing function

class ActFun_adp(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):  # input = membrane potential- threshold
        ctx.save_for_backward(input)
        return input.gt(0).float()  # is firing ???

    @staticmethod
    def backward(ctx, grad_output):  # approximate the gradients
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        # temp = abs(input) < lens
  
        if gradient_type == 'G':
            temp = torch.exp(-(input**2)/(2*lens**2))/torch.sqrt(2*torch.tensor(math.pi))/lens
        elif gradient_type == 'MG':
            temp = gaussian(input, mu=0., sigma=lens) * (1. + hight) \
                - gaussian(input, mu=lens, sigma=scale * lens) * hight \
                - gaussian(input, mu=-lens, sigma=scale * lens) * hight
        elif gradient_type =='linear':
            temp = F.relu(1-input.abs())
        elif gradient_type == 'slayer':
            temp = torch.exp(-5*input.abs())
        return grad_input * temp.float() * gamma

act_fun_adp = ActFun_adp.apply
# tau_m = torch.FloatTensor([tau_m])

def mem_update_adp(inputs, mem, spike, tau_adp, b, tau_m, dt=1, isAdapt=1):
    alpha = torch.exp(-1. * dt / tau_m).cuda()
    ro = torch.exp(-1. * dt / tau_adp).cuda()
    if isAdapt:
        beta = 1.8
    else:
        beta = 0.

    b = ro * b + (1 - ro) * spike
    B = b_j0 + beta * b
    mem = mem * alpha + (1 - alpha) * R_m * inputs - B * spike * dt
    
    inputs_ = mem - B
    spike = act_fun_adp(inputs_)  # act_fun : approximation firing function
    return mem, spike, B, b


def output_Neuron(inputs, mem, tau_m, dt=1):
    """
    The read out neuron is leaky integrator without spike
    """
    # alpha = torch.exp(-1. * dt / torch.FloatTensor([30.])).cuda()
    alpha = torch.exp(-1. * dt / tau_m).cuda()
    mem = mem * alpha + (1. - alpha) * R_m * inputs
    return mem


class RNN_custom(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN_custom, self).__init__()

        self.hidden_size = hidden_size
        # self.hidden_size = input_size
        self.i_2_h1 = nn.Linear(input_size, hidden_size[0])
        self.h1_2_h1 = nn.Linear(hidden_size[0], hidden_size[0])
        self.h1_2_h2 = nn.Linear(hidden_size[0], hidden_size[1])
        self.h2_2_h2 = nn.Linear(hidden_size[1], hidden_size[1])

        self.h2o = nn.Linear(hidden_size[1], output_size)

        self.tau_adp_h1 = nn.Parameter(torch.Tensor(hidden_size[0]))
        self.tau_adp_h2 = nn.Parameter(torch.Tensor(hidden_size[1]))
        self.tau_adp_o = nn.Parameter(torch.Tensor(output_size))
        self.tau_m_h1 = nn.Parameter(torch.Tensor(hidden_size[0]))
        self.tau_m_h2 = nn.Parameter(torch.Tensor(hidden_size[1]))
        self.tau_m_o = nn.Parameter(torch.Tensor(output_size))

        # nn.init.orthogonal_(self.h1_2_h1.weight)
        # nn.init.orthogonal_(self.h2_2_h2.weight)
        nn.init.orthogonal_(self.h1_2_h1.weight)
        nn.init.orthogonal_(self.h2_2_h2.weight)
        nn.init.xavier_uniform_(self.i_2_h1.weight)
        nn.init.xavier_uniform_(self.h1_2_h2.weight)
        nn.init.xavier_uniform_(self.h2o.weight)

        nn.init.constant_(self.i_2_h1.bias, 0)
        nn.init.constant_(self.h1_2_h2.bias, 0)
        nn.init.constant_(self.h2_2_h2.bias, 0)
        nn.init.constant_(self.h1_2_h1.bias, 0)

        nn.init.normal_(self.tau_adp_h1,150,10)
        nn.init.normal_(self.tau_adp_h2, 150,10)
        nn.init.normal_(self.tau_adp_o, 150,10)
        nn.init.normal_(self.tau_m_h1, 20.,5)
        nn.init.normal_(self.tau_m_h2, 20.,5)
        nn.init.normal_(self.tau_m_o, 20.,5)

        self.dp = nn.Dropout(0.1)

        self.b_h1 = self.b_h2 = self.b_o = 0

    def forward(self, input):
        batch_size, seq_num, input_dim = input.shape
        self.b_h1 = self.b_h2 = self.b_o = b_j0
        # mem_layer1 = spike_layer1 = torch.zeros(batch_size, self.hidden_size[0]).cuda()
        # mem_layer2 = spike_layer2 = torch.zeros(batch_size, self.hidden_size[1]).cuda()
        mem_layer1 = torch.rand(batch_size, self.hidden_size[0]).cuda()
        mem_layer2 = torch.rand(batch_size, self.hidden_size[1]).cuda()

        spike_layer1 = torch.zeros(batch_size, self.hidden_size[0]).cuda()
        spike_layer2 = torch.zeros(batch_size, self.hidden_size[1]).cuda()
        mem_output = torch.rand(batch_size, output_dim).cuda()
        output = torch.zeros(batch_size, output_dim).cuda()

        hidden_spike_ = []
        hidden_mem_ = []
        h2o_mem_ = []

        for i in range(seq_num):
            input_x = input[:, i, :]

            h_input = self.i_2_h1(input_x.float()) + self.h1_2_h1(spike_layer1)
            mem_layer1, spike_layer1, theta_h1, self.b_h1 = mem_update_adp(h_input, mem_layer1, spike_layer1,
                                                                         self.tau_adp_h1, self.b_h1,self.tau_m_h1)
            # spike_layer1 = self.dp(spike_layer1)
            h2_input = self.h1_2_h2(spike_layer1) + self.h2_2_h2(spike_layer2)
            mem_layer2, spike_layer2, theta_h2, self.b_h2 = mem_update_adp(h2_input, mem_layer2, spike_layer2,
                                                                         self.tau_adp_h2, self.b_h2, self.tau_m_h2)
            mem_output = output_Neuron(self.h2o(spike_layer2), mem_output, self.tau_m_o)
            if i > 10:
                output= output + F.softmax(mem_output, dim=1)#F.softmax(mem_output, dim=1)#

            hidden_spike_.append(spike_layer1.data.cpu().numpy())
            hidden_mem_.append(mem_layer1.data.cpu().numpy())
            h2o_mem_.append(mem_output.data.cpu().numpy())

        return output, hidden_spike_, hidden_mem_, h2o_mem_


'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 700
hidden_dim = [128,128]  # 128
output_dim = 20
seq_dim = 250  # Number of steps to unroll
num_encode = 700
total_steps = seq_dim

model = RNN_custom(input_dim, hidden_dim, output_dim)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model.to(device)
criterion = nn.CrossEntropyLoss()
learning_rate =  1e-2

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,eps=1e-5)

# base_params = [model.i_2_h1.weight, model.i_2_h1.bias,
#                model.h1_2_h1.weight, model.h1_2_h1.bias,
#                model.h1_2_h2.weight, model.h1_2_h2.bias,
#                model.h2_2_h2.weight, model.h2_2_h2.bias,
#                model.h2o.weight, model.h2o.bias]
# optimizer = torch.optim.Adam([
#     {'params': base_params},
#     {'params': model.tau_adp_h1, 'lr': learning_rate * 5},
#     {'params': model.tau_adp_h2, 'lr': learning_rate * 5},
#     {'params': model.tau_m_h1, 'lr': learning_rate * 1},
#     {'params': model.tau_m_h2, 'lr': learning_rate * 1},
#     {'params': model.tau_m_o, 'lr': learning_rate * 1}],
#     lr=learning_rate,eps=1e-5)

scheduler = StepLR(optimizer, step_size=10, gamma=.5)

path = os.path.join(CHECKPOINT_PATH,'bojian_model')  # .pth'

def train(model, num_epochs=150):
    acc = []
    best_accuracy = 80
    
    
    
    for epoch in range(1,num_epochs):
        loss_sum = 0
        total = 0
        correct = 0
        for i, (images, labels) in enumerate(train_loader):
            #images = images.view(-1, seq_dim, input_dim).requires_grad_().to(device)
            images = images.view(-1, seq_dim, input_dim).to(device)
            _, labels = torch.max(labels.data, 1)
            labels = labels.long().to(device)
            # Clear gradients w.r.t. parameters
            optimizer.zero_grad()
            # Forward pass to get output/logits
            outputs, _,_,_ = model(images)
            _, predicted = torch.max(outputs.data, 1)
            # Calculate Loss: softmax --> cross entropy loss
            loss = criterion(outputs, labels)
            loss_sum+= loss
            # Getting gradients w.r.t. parameters
            loss.backward()
            # Updating parameters
            optimizer.step()

            total += labels.size(0)
            if torch.cuda.is_available():
                correct += (predicted.cpu() == labels.long().cpu()).sum()
            else:
                correct += (predicted == labels).sum()

        scheduler.step()
        accuracy = 100. * correct.numpy() / total
        # accuracy,_ = test(model, train_loader)
        ts_acc,fr = test(model,is_test=0)
        if ts_acc > best_accuracy and accuracy > 80:
            torch.save(model, path+'/rsnn{}_alif_new_epoch{}.pth'.format(hidden_dim[0], epoch))
            best_accuracy = ts_acc
 
        print('epoch: ', epoch, '. Loss: ', loss.item(), '. Tr Accuracy: ', accuracy, '. Ts Accuracy: ',
         ts_acc, 'Fr: ',fr)

        acc.append(accuracy)
        # if epoch %5==0:
        #     print('epoch: ', epoch, '. Loss: ', loss_sum.item()/i, 
        #             '. Tr Accuracy: ', accuracy, '. Ts Accuracy: ', ts_acc,', Fr: ',fr)
    return acc


def test(model, dataloader=test_loader,is_test=0):
    correct = 0
    total = 0
    # Iterate through test dataset
    for images, labels in dataloader:
        images = images.view(-1, seq_dim, input_dim).to(device)
        _, labels = torch.max(labels.data, 1)

        outputs, fr_,_,_ = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        if torch.cuda.is_available():
            correct += (predicted.cpu() == labels.long().cpu()).sum()
        else:
            correct += (predicted == labels).sum()

    accuracy = 100. * correct.numpy() / total
    if is_test:
        print('Mean FR: ', np.array(fr_).mean())
    return accuracy, np.array(fr_).mean()

# dataset shape:  (8156, 250, 700)
# dataset shape:  (2264, 250, 700)
# gradient_type:  MG
# hight:  0.15 ;scale:  6.0
# device: cuda:0
# epoch:  1 . Loss:  1.4047224521636963 . Tr Accuracy:  30.946542422756252 . Ts Accuracy:  40.90106007067138 Fr:  0.10361849
# epoch:  2 . Loss:  0.5009759664535522 . Tr Accuracy:  58.26385483079941 . Ts Accuracy:  75.79505300353357 Fr:  0.14015365
# epoch:  3 . Loss:  0.8235954642295837 . Tr Accuracy:  80.44384502206964 . Ts Accuracy:  86.17491166077738 Fr:  0.12855339
# epoch:  4 . Loss:  0.1801595240831375 . Tr Accuracy:  85.93673369298676 . Ts Accuracy:  78.00353356890459 Fr:  0.12772396
# epoch:  5 . Loss:  0.6335716843605042 . Tr Accuracy:  87.84943599803826 . Ts Accuracy:  83.25971731448763 Fr:  0.122699216
# epoch:  6 . Loss:  0.4184652864933014 . Tr Accuracy:  88.46248160863168 . Ts Accuracy:  85.02650176678445 Fr:  0.12641016
# epoch:  7 . Loss:  0.11178665608167648 . Tr Accuracy:  91.66257969592938 . Ts Accuracy:  83.70141342756183 Fr:  0.11583985
# epoch:  8 . Loss:  0.24507765471935272 . Tr Accuracy:  93.80823933300637 . Ts Accuracy:  80.43286219081273 Fr:  0.11689453
# epoch:  9 . Loss:  0.02580219879746437 . Tr Accuracy:  93.6856302108877 . Ts Accuracy:  78.53356890459364 Fr:  0.11530859
# epoch:  10 . Loss:  0.132295161485672 . Tr Accuracy:  95.15693967631192 . Ts Accuracy:  87.0583038869258 Fr:  0.107161455
# epoch:  11 . Loss:  0.052720583975315094 . Tr Accuracy:  97.4619911721432 . Ts Accuracy:  87.54416961130742 Fr:  0.111710936
# epoch:  12 . Loss:  0.013912809081375599 . Tr Accuracy:  97.63364394310936 . Ts Accuracy:  88.29505300353357 Fr:  0.11046224
# epoch:  13 . Loss:  0.013542967848479748 . Tr Accuracy:  98.43060323688083 . Ts Accuracy:  84.31978798586573 Fr:  0.11176432
# epoch:  14 . Loss:  0.03696903958916664 . Tr Accuracy:  98.14860225600785 . Ts Accuracy:  85.95406360424029 Fr:  0.113291666
# epoch:  15 . Loss:  0.12882192432880402 . Tr Accuracy:  98.4428641490927 . Ts Accuracy:  84.67314487632508 Fr:  0.11321094
# epoch:  16 . Loss:  0.008622714318335056 . Tr Accuracy:  98.60225600784699 . Ts Accuracy:  87.72084805653711 Fr:  0.11448698
# epoch:  17 . Loss:  0.08007299154996872 . Tr Accuracy:  98.58999509563512 . Ts Accuracy:  85.07067137809187 Fr:  0.109397136
# epoch:  18 . Loss:  0.17344336211681366 . Tr Accuracy:  97.91564492398234 . Ts Accuracy:  86.43992932862191 Fr:  0.114264324
# epoch:  19 . Loss:  0.31412410736083984 . Tr Accuracy:  97.89112309955861 . Ts Accuracy:  90.68021201413427 Fr:  0.110308595
# Mean FR:  0.10995052
#  Accuracy:  90.7243816254417


gradient_type:  MG
hight:  0.15 ;scale:  6.0
device: cuda:0


In [4]:
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 700
hidden_dim = [128,128]  # 128
output_dim = 20
seq_dim = 250  # Number of steps to unroll
num_encode = 700
total_steps = seq_dim

model = RNN_custom(input_dim, hidden_dim, output_dim)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model.to(device)
criterion = nn.CrossEntropyLoss()
learning_rate =  1e-2

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,eps=1e-5)

# base_params = [model.i_2_h1.weight, model.i_2_h1.bias,
#                model.h1_2_h1.weight, model.h1_2_h1.bias,
#                model.h1_2_h2.weight, model.h1_2_h2.bias,
#                model.h2_2_h2.weight, model.h2_2_h2.bias,
#                model.h2o.weight, model.h2o.bias]
# optimizer = torch.optim.Adam([
#     {'params': base_params},
#     {'params': model.tau_adp_h1, 'lr': learning_rate * 5},
#     {'params': model.tau_adp_h2, 'lr': learning_rate * 5},
#     {'params': model.tau_m_h1, 'lr': learning_rate * 1},
#     {'params': model.tau_m_h2, 'lr': learning_rate * 1},
#     {'params': model.tau_m_o, 'lr': learning_rate * 1}],
#     lr=learning_rate,eps=1e-5)
scheduler = StepLR(optimizer, step_size=10, gamma=.5)

path = os.path.join(CHECKPOINT_PATH,'bojian_model')  # .pth'

acc = train(model, num_epochs)
test_acc,fr = test(model,is_test=1)
print(' Accuracy: ', test_acc)


device: cuda:0
epoch:  1 . Loss:  1.1417795419692993 . Tr Accuracy:  28.506397637795274 . Ts Accuracy:  61.42857142857143 Fr:  0.12849511
epoch:  2 . Loss:  0.5131941437721252 . Tr Accuracy:  67.39665354330708 . Ts Accuracy:  71.96428571428571 Fr:  0.119453125
epoch:  3 . Loss:  0.44096359610557556 . Tr Accuracy:  82.45570866141732 . Ts Accuracy:  75.71428571428571 Fr:  0.11837598
epoch:  4 . Loss:  0.16837456822395325 . Tr Accuracy:  89.0009842519685 . Ts Accuracy:  81.33928571428571 Fr:  0.11596582
epoch:  5 . Loss:  0.27148526906967163 . Tr Accuracy:  92.00295275590551 . Ts Accuracy:  74.0625 Fr:  0.115331054
epoch:  6 . Loss:  0.2539396286010742 . Tr Accuracy:  94.04527559055119 . Ts Accuracy:  77.99107142857143 Fr:  0.12006006
epoch:  7 . Loss:  0.17388473451137543 . Tr Accuracy:  93.31938976377953 . Ts Accuracy:  82.54464285714286 Fr:  0.11574658
epoch:  8 . Loss:  0.20907920598983765 . Tr Accuracy:  95.76771653543307 . Ts Accuracy:  84.73214285714286 Fr:  0.11130713
epoch:  9 . 

### images>0, dropout 0.2

In [5]:
dropout = torch.nn.Dropout(p=0.2, inplace=False)

def train(model, num_epochs=150):
    acc = []
    best_accuracy = 80
    
    for epoch in range(1,num_epochs):
        loss_sum = 0
        total = 0
        correct = 0
        for i, (images, labels) in enumerate(train_loader):
            #images = images.view(-1, seq_dim, input_dim).requires_grad_().to(device)

            images = dropout(images.float())
            images = images > 0            
            
            images = images.view(-1, seq_dim, input_dim).to(device)
            _, labels = torch.max(labels.data, 1)
            labels = labels.long().to(device)
            # Clear gradients w.r.t. parameters
            optimizer.zero_grad()
            # Forward pass to get output/logits
            outputs, _,_,_ = model(images)
            _, predicted = torch.max(outputs.data, 1)
            # Calculate Loss: softmax --> cross entropy loss
            loss = criterion(outputs, labels)
            loss_sum+= loss
            # Getting gradients w.r.t. parameters
            loss.backward()
            # Updating parameters
            optimizer.step()

            total += labels.size(0)
            if torch.cuda.is_available():
                correct += (predicted.cpu() == labels.long().cpu()).sum()
            else:
                correct += (predicted == labels).sum()

        scheduler.step()
        accuracy = 100. * correct.numpy() / total
        # accuracy,_ = test(model, train_loader)
        ts_acc,fr = test(model,is_test=0)
        if ts_acc > best_accuracy and accuracy > 80:
            torch.save(model, path+'/rsnn{}_alif_new_epoch{}.pth'.format(hidden_dim[0], epoch))
            best_accuracy = ts_acc
 
        print('epoch: ', epoch, '. Loss: ', loss.item(), '. Tr Accuracy: ', accuracy, '. Ts Accuracy: ',
         ts_acc, 'Fr: ',fr)

        acc.append(accuracy)
        # if epoch %5==0:
        #     print('epoch: ', epoch, '. Loss: ', loss_sum.item()/i, 
        #             '. Tr Accuracy: ', accuracy, '. Ts Accuracy: ', ts_acc,', Fr: ',fr)
    return acc


def test(model, dataloader=test_loader,is_test=0):
    correct = 0
    total = 0
    # Iterate through test dataset
    for images, labels in dataloader:
        
        images = dropout(images.float())
        images = images > 0    
        
        images = images.view(-1, seq_dim, input_dim).to(device)
        _, labels = torch.max(labels.data, 1)

        outputs, fr_,_,_ = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        if torch.cuda.is_available():
            correct += (predicted.cpu() == labels.long().cpu()).sum()
        else:
            correct += (predicted == labels).sum()

    accuracy = 100. * correct.numpy() / total
    if is_test:
        print('Mean FR: ', np.array(fr_).mean())
    return accuracy, np.array(fr_).mean()


'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 700
hidden_dim = [128,128]  # 128
output_dim = 20
seq_dim = 250  # Number of steps to unroll
num_encode = 700
total_steps = seq_dim

model = RNN_custom(input_dim, hidden_dim, output_dim)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model.to(device)
criterion = nn.CrossEntropyLoss()
learning_rate =  1e-2

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,eps=1e-5)

# base_params = [model.i_2_h1.weight, model.i_2_h1.bias,
#                model.h1_2_h1.weight, model.h1_2_h1.bias,
#                model.h1_2_h2.weight, model.h1_2_h2.bias,
#                model.h2_2_h2.weight, model.h2_2_h2.bias,
#                model.h2o.weight, model.h2o.bias]
# optimizer = torch.optim.Adam([
#     {'params': base_params},
#     {'params': model.tau_adp_h1, 'lr': learning_rate * 5},
#     {'params': model.tau_adp_h2, 'lr': learning_rate * 5},
#     {'params': model.tau_m_h1, 'lr': learning_rate * 1},
#     {'params': model.tau_m_h2, 'lr': learning_rate * 1},
#     {'params': model.tau_m_o, 'lr': learning_rate * 1}],
#     lr=learning_rate,eps=1e-5)

scheduler = StepLR(optimizer, step_size=10, gamma=.5)

path = os.path.join(CHECKPOINT_PATH,'bojian_model')  # .pth'

acc = train(model, num_epochs)
test_acc,fr = test(model,is_test=1)
print(' Accuracy: ', test_acc)

device: cuda:0
epoch:  1 . Loss:  1.2821640968322754 . Tr Accuracy:  26.85777559055118 . Ts Accuracy:  66.38392857142857 Fr:  0.12793213
epoch:  2 . Loss:  0.4739812910556793 . Tr Accuracy:  70.3125 . Ts Accuracy:  75.3125 Fr:  0.116929196
epoch:  3 . Loss:  0.42120224237442017 . Tr Accuracy:  83.14468503937007 . Ts Accuracy:  79.91071428571429 Fr:  0.12220996
epoch:  4 . Loss:  0.26949840784072876 . Tr Accuracy:  87.37696850393701 . Ts Accuracy:  82.99107142857143 Fr:  0.12340674
epoch:  5 . Loss:  0.08047826588153839 . Tr Accuracy:  91.21555118110236 . Ts Accuracy:  80.44642857142857 Fr:  0.120833986
epoch:  6 . Loss:  0.2110244631767273 . Tr Accuracy:  91.96604330708661 . Ts Accuracy:  84.59821428571429 Fr:  0.1158706
epoch:  7 . Loss:  0.17956425249576569 . Tr Accuracy:  93.97145669291339 . Ts Accuracy:  82.58928571428571 Fr:  0.120810546
epoch:  8 . Loss:  0.15149128437042236 . Tr Accuracy:  93.9099409448819 . Ts Accuracy:  76.875 Fr:  0.11675781
epoch:  9 . Loss:  0.1048161238431

In [4]:
dropout = torch.nn.Dropout(p=0.2, inplace=False)

def train(model, num_epochs=150):
    acc = []
    best_accuracy = 80
    
    for epoch in range(1,num_epochs):
        loss_sum = 0
        total = 0
        correct = 0
        for i, (images, labels) in enumerate(train_loader):
            #images = images.view(-1, seq_dim, input_dim).requires_grad_().to(device)

            images = dropout(images.float())
            images = images > 0            
            
            images = images.view(-1, seq_dim, input_dim).to(device)
            _, labels = torch.max(labels.data, 1)
            labels = labels.long().to(device)
            # Clear gradients w.r.t. parameters
            optimizer.zero_grad()
            # Forward pass to get output/logits
            outputs, _,_,_ = model(images)
            _, predicted = torch.max(outputs.data, 1)
            # Calculate Loss: softmax --> cross entropy loss
            loss = criterion(outputs, labels)
            loss_sum+= loss
            # Getting gradients w.r.t. parameters
            loss.backward()
            # Updating parameters
            optimizer.step()

            total += labels.size(0)
            if torch.cuda.is_available():
                correct += (predicted.cpu() == labels.long().cpu()).sum()
            else:
                correct += (predicted == labels).sum()

        scheduler.step()
        accuracy = 100. * correct.numpy() / total
        # accuracy,_ = test(model, train_loader)
        ts_acc,fr = test(model,is_test=0)
        if ts_acc > best_accuracy and accuracy > 80:
            torch.save(model, path+'/rsnn{}_alif_new_epoch{}.pth'.format(hidden_dim[0], epoch))
            best_accuracy = ts_acc
 
        print('epoch: ', epoch, '. Loss: ', loss.item(), '. Tr Accuracy: ', accuracy, '. Ts Accuracy: ',
         ts_acc, 'Fr: ',fr)

        acc.append(accuracy)
        # if epoch %5==0:
        #     print('epoch: ', epoch, '. Loss: ', loss_sum.item()/i, 
        #             '. Tr Accuracy: ', accuracy, '. Ts Accuracy: ', ts_acc,', Fr: ',fr)
    return acc


def test(model, dataloader=test_loader,is_test=0):
    correct = 0
    total = 0
    # Iterate through test dataset
    for images, labels in dataloader:
        
        images = dropout(images.float())
        images = images > 0    
        
        images = images.view(-1, seq_dim, input_dim).to(device)
        _, labels = torch.max(labels.data, 1)

        outputs, fr_,_,_ = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        if torch.cuda.is_available():
            correct += (predicted.cpu() == labels.long().cpu()).sum()
        else:
            correct += (predicted == labels).sum()

    accuracy = 100. * correct.numpy() / total
    if is_test:
        print('Mean FR: ', np.array(fr_).mean())
    return accuracy, np.array(fr_).mean()


'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 700
hidden_dim = [128,128]  # 128
output_dim = 20
seq_dim = 250  # Number of steps to unroll
num_encode = 700
total_steps = seq_dim

model = RNN_custom(input_dim, hidden_dim, output_dim)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model.to(device)
criterion = nn.CrossEntropyLoss()
learning_rate =  1e-2

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,eps=1e-5)

# base_params = [model.i_2_h1.weight, model.i_2_h1.bias,
#                model.h1_2_h1.weight, model.h1_2_h1.bias,
#                model.h1_2_h2.weight, model.h1_2_h2.bias,
#                model.h2_2_h2.weight, model.h2_2_h2.bias,
#                model.h2o.weight, model.h2o.bias]
# optimizer = torch.optim.Adam([
#     {'params': base_params},
#     {'params': model.tau_adp_h1, 'lr': learning_rate * 5},
#     {'params': model.tau_adp_h2, 'lr': learning_rate * 5},
#     {'params': model.tau_m_h1, 'lr': learning_rate * 1},
#     {'params': model.tau_m_h2, 'lr': learning_rate * 1},
#     {'params': model.tau_m_o, 'lr': learning_rate * 1}],
#     lr=learning_rate,eps=1e-5)

scheduler = StepLR(optimizer, step_size=10, gamma=.5)

path = os.path.join(CHECKPOINT_PATH,'bojian_model')  # .pth'

acc = train(model, num_epochs)
test_acc,fr = test(model,is_test=1)
print(' Accuracy: ', test_acc)

device: cuda:0
epoch:  1 . Loss:  1.0533266067504883 . Tr Accuracy:  28.59251968503937 . Ts Accuracy:  61.07142857142857 Fr:  0.13657813
epoch:  2 . Loss:  0.6155337691307068 . Tr Accuracy:  69.91879921259843 . Ts Accuracy:  71.69642857142857 Fr:  0.13449512
epoch:  3 . Loss:  0.7312968969345093 . Tr Accuracy:  81.2992125984252 . Ts Accuracy:  75.22321428571429 Fr:  0.12997754
epoch:  4 . Loss:  0.40226882696151733 . Tr Accuracy:  87.09399606299213 . Ts Accuracy:  78.21428571428571 Fr:  0.1302876
epoch:  5 . Loss:  0.0942985787987709 . Tr Accuracy:  89.18553149606299 . Ts Accuracy:  80.08928571428571 Fr:  0.123591796
epoch:  6 . Loss:  0.2178582102060318 . Tr Accuracy:  92.0275590551181 . Ts Accuracy:  84.77678571428571 Fr:  0.12568799
epoch:  7 . Loss:  0.13728997111320496 . Tr Accuracy:  94.41437007874016 . Ts Accuracy:  82.23214285714286 Fr:  0.11818018
epoch:  8 . Loss:  0.04252934083342552 . Tr Accuracy:  94.73425196850394 . Ts Accuracy:  85.84821428571429 Fr:  0.12506787
epoch:  

In [5]:
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 700
hidden_dim = [64,64]  # 128
output_dim = 20
seq_dim = 250  # Number of steps to unroll
num_encode = 700
total_steps = seq_dim

model = RNN_custom(input_dim, hidden_dim, output_dim)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model.to(device)
criterion = nn.CrossEntropyLoss()
learning_rate =  1e-2

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,eps=1e-5)

# base_params = [model.i_2_h1.weight, model.i_2_h1.bias,
#                model.h1_2_h1.weight, model.h1_2_h1.bias,
#                model.h1_2_h2.weight, model.h1_2_h2.bias,
#                model.h2_2_h2.weight, model.h2_2_h2.bias,
#                model.h2o.weight, model.h2o.bias]
# optimizer = torch.optim.Adam([
#     {'params': base_params},
#     {'params': model.tau_adp_h1, 'lr': learning_rate * 5},
#     {'params': model.tau_adp_h2, 'lr': learning_rate * 5},
#     {'params': model.tau_m_h1, 'lr': learning_rate * 1},
#     {'params': model.tau_m_h2, 'lr': learning_rate * 1},
#     {'params': model.tau_m_o, 'lr': learning_rate * 1}],
#     lr=learning_rate,eps=1e-5)

scheduler = StepLR(optimizer, step_size=10, gamma=.5)

path = os.path.join(CHECKPOINT_PATH,'bojian_model')  # .pth'

acc = train(model, num_epochs)
test_acc,fr = test(model,is_test=1)
print(' Accuracy: ', test_acc)

device: cuda:0
epoch:  1 . Loss:  1.6024893522262573 . Tr Accuracy:  21.24753937007874 . Ts Accuracy:  49.642857142857146 Fr:  0.14380664
epoch:  2 . Loss:  1.101971983909607 . Tr Accuracy:  57.04970472440945 . Ts Accuracy:  67.5 Fr:  0.16943066
epoch:  3 . Loss:  0.7828410267829895 . Tr Accuracy:  75.49212598425197 . Ts Accuracy:  72.99107142857143 Fr:  0.16090918
epoch:  4 . Loss:  0.4361381530761719 . Tr Accuracy:  83.23080708661418 . Ts Accuracy:  74.64285714285714 Fr:  0.16295801
epoch:  5 . Loss:  0.4594704806804657 . Tr Accuracy:  88.16437007874016 . Ts Accuracy:  80.49107142857143 Fr:  0.16607226
epoch:  6 . Loss:  0.6766268610954285 . Tr Accuracy:  91.64616141732283 . Ts Accuracy:  79.15178571428571 Fr:  0.16194433
epoch:  7 . Loss:  0.4412130117416382 . Tr Accuracy:  92.61811023622047 . Ts Accuracy:  84.41964285714286 Fr:  0.15918067
epoch:  8 . Loss:  0.26646819710731506 . Tr Accuracy:  93.27017716535433 . Ts Accuracy:  81.5625 Fr:  0.16016406
epoch:  9 . Loss:  0.1421553045

In [6]:
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 700
hidden_dim = [48,48]  # 128
output_dim = 20
seq_dim = 250  # Number of steps to unroll
num_encode = 700
total_steps = seq_dim

model = RNN_custom(input_dim, hidden_dim, output_dim)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model.to(device)
criterion = nn.CrossEntropyLoss()
learning_rate =  1e-2

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,eps=1e-5)

# base_params = [model.i_2_h1.weight, model.i_2_h1.bias,
#                model.h1_2_h1.weight, model.h1_2_h1.bias,
#                model.h1_2_h2.weight, model.h1_2_h2.bias,
#                model.h2_2_h2.weight, model.h2_2_h2.bias,
#                model.h2o.weight, model.h2o.bias]
# optimizer = torch.optim.Adam([
#     {'params': base_params},
#     {'params': model.tau_adp_h1, 'lr': learning_rate * 5},
#     {'params': model.tau_adp_h2, 'lr': learning_rate * 5},
#     {'params': model.tau_m_h1, 'lr': learning_rate * 1},
#     {'params': model.tau_m_h2, 'lr': learning_rate * 1},
#     {'params': model.tau_m_o, 'lr': learning_rate * 1}],
#     lr=learning_rate,eps=1e-5)

scheduler = StepLR(optimizer, step_size=10, gamma=.5)

path = os.path.join(CHECKPOINT_PATH,'bojian_model')  # .pth'

acc = train(model, num_epochs)
test_acc,fr = test(model,is_test=1)
print(' Accuracy: ', test_acc)

device: cuda:0
epoch:  1 . Loss:  1.7800135612487793 . Tr Accuracy:  17.753444881889763 . Ts Accuracy:  36.294642857142854 Fr:  0.15179297
epoch:  2 . Loss:  0.7516031861305237 . Tr Accuracy:  58.87057086614173 . Ts Accuracy:  68.83928571428571 Fr:  0.16226692
epoch:  3 . Loss:  0.5555938482284546 . Tr Accuracy:  75.99655511811024 . Ts Accuracy:  77.72321428571429 Fr:  0.16349609
epoch:  4 . Loss:  0.3401418924331665 . Tr Accuracy:  82.8248031496063 . Ts Accuracy:  73.52678571428571 Fr:  0.16874349
epoch:  5 . Loss:  0.5352290272712708 . Tr Accuracy:  87.37696850393701 . Ts Accuracy:  67.1875 Fr:  0.16528125
epoch:  6 . Loss:  0.29107001423835754 . Tr Accuracy:  90.45275590551181 . Ts Accuracy:  76.16071428571429 Fr:  0.17019661
epoch:  7 . Loss:  0.37120315432548523 . Tr Accuracy:  91.57234251968504 . Ts Accuracy:  72.8125 Fr:  0.16496615
epoch:  8 . Loss:  0.12378918379545212 . Tr Accuracy:  93.29478346456693 . Ts Accuracy:  74.95535714285714 Fr:  0.17080729
epoch:  9 . Loss:  0.2088

In [7]:
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 700
hidden_dim = [32,32]  # 128
output_dim = 20
seq_dim = 250  # Number of steps to unroll
num_encode = 700
total_steps = seq_dim

model = RNN_custom(input_dim, hidden_dim, output_dim)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model.to(device)
criterion = nn.CrossEntropyLoss()
learning_rate =  1e-2

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,eps=1e-5)

# base_params = [model.i_2_h1.weight, model.i_2_h1.bias,
#                model.h1_2_h1.weight, model.h1_2_h1.bias,
#                model.h1_2_h2.weight, model.h1_2_h2.bias,
#                model.h2_2_h2.weight, model.h2_2_h2.bias,
#                model.h2o.weight, model.h2o.bias]
# optimizer = torch.optim.Adam([
#     {'params': base_params},
#     {'params': model.tau_adp_h1, 'lr': learning_rate * 5},
#     {'params': model.tau_adp_h2, 'lr': learning_rate * 5},
#     {'params': model.tau_m_h1, 'lr': learning_rate * 1},
#     {'params': model.tau_m_h2, 'lr': learning_rate * 1},
#     {'params': model.tau_m_o, 'lr': learning_rate * 1}],
#     lr=learning_rate,eps=1e-5)

scheduler = StepLR(optimizer, step_size=10, gamma=.5)

path = os.path.join(CHECKPOINT_PATH,'bojian_model')  # .pth'

acc = train(model, num_epochs)
test_acc,fr = test(model,is_test=1)
print(' Accuracy: ', test_acc)

device: cuda:0
epoch:  1 . Loss:  2.2305121421813965 . Tr Accuracy:  11.700295275590552 . Ts Accuracy:  23.571428571428573 Fr:  0.20851563
epoch:  2 . Loss:  1.247761845588684 . Tr Accuracy:  38.45964566929134 . Ts Accuracy:  45.580357142857146 Fr:  0.18563086
epoch:  3 . Loss:  1.0206964015960693 . Tr Accuracy:  59.30118110236221 . Ts Accuracy:  64.41964285714286 Fr:  0.1778418
epoch:  4 . Loss:  0.8707325458526611 . Tr Accuracy:  71.56742125984252 . Ts Accuracy:  70.71428571428571 Fr:  0.18417577
epoch:  5 . Loss:  0.3953717052936554 . Tr Accuracy:  77.79281496062993 . Ts Accuracy:  67.41071428571429 Fr:  0.18475977
epoch:  6 . Loss:  0.48988014459609985 . Tr Accuracy:  83.30462598425197 . Ts Accuracy:  68.97321428571429 Fr:  0.18675
epoch:  7 . Loss:  0.415967732667923 . Tr Accuracy:  85.35925196850394 . Ts Accuracy:  70.625 Fr:  0.1925918
epoch:  8 . Loss:  0.345842182636261 . Tr Accuracy:  87.80757874015748 . Ts Accuracy:  75.625 Fr:  0.19554493
epoch:  9 . Loss:  0.27550792694091

In [8]:
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 700
hidden_dim = [112,112]  # 128
output_dim = 20
seq_dim = 250  # Number of steps to unroll
num_encode = 700
total_steps = seq_dim

model = RNN_custom(input_dim, hidden_dim, output_dim)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model.to(device)
criterion = nn.CrossEntropyLoss()
learning_rate =  1e-2

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,eps=1e-5)

# base_params = [model.i_2_h1.weight, model.i_2_h1.bias,
#                model.h1_2_h1.weight, model.h1_2_h1.bias,
#                model.h1_2_h2.weight, model.h1_2_h2.bias,
#                model.h2_2_h2.weight, model.h2_2_h2.bias,
#                model.h2o.weight, model.h2o.bias]
# optimizer = torch.optim.Adam([
#     {'params': base_params},
#     {'params': model.tau_adp_h1, 'lr': learning_rate * 5},
#     {'params': model.tau_adp_h2, 'lr': learning_rate * 5},
#     {'params': model.tau_m_h1, 'lr': learning_rate * 1},
#     {'params': model.tau_m_h2, 'lr': learning_rate * 1},
#     {'params': model.tau_m_o, 'lr': learning_rate * 1}],
#     lr=learning_rate,eps=1e-5)

scheduler = StepLR(optimizer, step_size=10, gamma=.5)

path = os.path.join(CHECKPOINT_PATH,'bojian_model')  # .pth'

acc = train(model, num_epochs)
test_acc,fr = test(model,is_test=1)
print(' Accuracy: ', test_acc)

device: cuda:0
epoch:  1 . Loss:  1.8285083770751953 . Tr Accuracy:  25.93503937007874 . Ts Accuracy:  47.99107142857143 Fr:  0.14043806
epoch:  2 . Loss:  0.5858820676803589 . Tr Accuracy:  66.43700787401575 . Ts Accuracy:  75.44642857142857 Fr:  0.14046875
epoch:  3 . Loss:  0.2967982888221741 . Tr Accuracy:  83.13238188976378 . Ts Accuracy:  74.33035714285714 Fr:  0.13985212
epoch:  4 . Loss:  0.29008811712265015 . Tr Accuracy:  87.73375984251969 . Ts Accuracy:  78.57142857142857 Fr:  0.13194476
epoch:  5 . Loss:  0.20904408395290375 . Tr Accuracy:  90.41584645669292 . Ts Accuracy:  82.54464285714286 Fr:  0.13089676
epoch:  6 . Loss:  0.12574677169322968 . Tr Accuracy:  92.51968503937007 . Ts Accuracy:  78.97321428571429 Fr:  0.12667857
epoch:  7 . Loss:  0.2575814723968506 . Tr Accuracy:  94.52509842519684 . Ts Accuracy:  86.78571428571429 Fr:  0.1280731
epoch:  8 . Loss:  0.18737050890922546 . Tr Accuracy:  94.73425196850394 . Ts Accuracy:  80.80357142857143 Fr:  0.12865067
epoch:

In [9]:
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 700
hidden_dim = [96,96]  # 128
output_dim = 20
seq_dim = 250  # Number of steps to unroll
num_encode = 700
total_steps = seq_dim

model = RNN_custom(input_dim, hidden_dim, output_dim)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model.to(device)
criterion = nn.CrossEntropyLoss()
learning_rate =  1e-2

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,eps=1e-5)

# base_params = [model.i_2_h1.weight, model.i_2_h1.bias,
#                model.h1_2_h1.weight, model.h1_2_h1.bias,
#                model.h1_2_h2.weight, model.h1_2_h2.bias,
#                model.h2_2_h2.weight, model.h2_2_h2.bias,
#                model.h2o.weight, model.h2o.bias]
# optimizer = torch.optim.Adam([
#     {'params': base_params},
#     {'params': model.tau_adp_h1, 'lr': learning_rate * 5},
#     {'params': model.tau_adp_h2, 'lr': learning_rate * 5},
#     {'params': model.tau_m_h1, 'lr': learning_rate * 1},
#     {'params': model.tau_m_h2, 'lr': learning_rate * 1},
#     {'params': model.tau_m_o, 'lr': learning_rate * 1}],
#     lr=learning_rate,eps=1e-5)

scheduler = StepLR(optimizer, step_size=10, gamma=.5)

path = os.path.join(CHECKPOINT_PATH,'bojian_model')  # .pth'

acc = train(model, num_epochs)
test_acc,fr = test(model,is_test=1)
print(' Accuracy: ', test_acc)

device: cuda:0
epoch:  1 . Loss:  1.6252371072769165 . Tr Accuracy:  21.838090551181104 . Ts Accuracy:  40.669642857142854 Fr:  0.1609069
epoch:  2 . Loss:  1.2593910694122314 . Tr Accuracy:  52.09153543307087 . Ts Accuracy:  57.36607142857143 Fr:  0.15891472
epoch:  3 . Loss:  1.0506930351257324 . Tr Accuracy:  67.67962598425197 . Ts Accuracy:  65.3125 Fr:  0.15386589
epoch:  4 . Loss:  0.4382389783859253 . Tr Accuracy:  76.63631889763779 . Ts Accuracy:  71.91964285714286 Fr:  0.1525651
epoch:  5 . Loss:  0.34050437808036804 . Tr Accuracy:  82.9109251968504 . Ts Accuracy:  76.60714285714286 Fr:  0.14895703
epoch:  6 . Loss:  0.6541894674301147 . Tr Accuracy:  85.80216535433071 . Ts Accuracy:  73.25892857142857 Fr:  0.15017448
epoch:  7 . Loss:  0.502514660358429 . Tr Accuracy:  88.85334645669292 . Ts Accuracy:  75.49107142857143 Fr:  0.14427409
epoch:  8 . Loss:  0.2159011960029602 . Tr Accuracy:  90.69881889763779 . Ts Accuracy:  79.59821428571429 Fr:  0.1401263
epoch:  9 . Loss:  0.

In [10]:
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 700
hidden_dim = [80,80]  # 128
output_dim = 20
seq_dim = 250  # Number of steps to unroll
num_encode = 700
total_steps = seq_dim

model = RNN_custom(input_dim, hidden_dim, output_dim)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model.to(device)
criterion = nn.CrossEntropyLoss()
learning_rate =  1e-2

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,eps=1e-5)

# base_params = [model.i_2_h1.weight, model.i_2_h1.bias,
#                model.h1_2_h1.weight, model.h1_2_h1.bias,
#                model.h1_2_h2.weight, model.h1_2_h2.bias,
#                model.h2_2_h2.weight, model.h2_2_h2.bias,
#                model.h2o.weight, model.h2o.bias]
# optimizer = torch.optim.Adam([
#     {'params': base_params},
#     {'params': model.tau_adp_h1, 'lr': learning_rate * 5},
#     {'params': model.tau_adp_h2, 'lr': learning_rate * 5},
#     {'params': model.tau_m_h1, 'lr': learning_rate * 1},
#     {'params': model.tau_m_h2, 'lr': learning_rate * 1},
#     {'params': model.tau_m_o, 'lr': learning_rate * 1}],
#     lr=learning_rate,eps=1e-5)

scheduler = StepLR(optimizer, step_size=10, gamma=.5)

path = os.path.join(CHECKPOINT_PATH,'bojian_model')  # .pth'

acc = train(model, num_epochs)
test_acc,fr = test(model,is_test=1)
print(' Accuracy: ', test_acc)

device: cuda:0
epoch:  1 . Loss:  1.5336837768554688 . Tr Accuracy:  23.351377952755904 . Ts Accuracy:  47.36607142857143 Fr:  0.14830548
epoch:  2 . Loss:  1.0515326261520386 . Tr Accuracy:  62.463090551181104 . Ts Accuracy:  68.25892857142857 Fr:  0.13543594
epoch:  3 . Loss:  0.6928795576095581 . Tr Accuracy:  75.1599409448819 . Ts Accuracy:  79.55357142857143 Fr:  0.13739297
epoch:  4 . Loss:  0.44312426447868347 . Tr Accuracy:  84.81791338582677 . Ts Accuracy:  80.3125 Fr:  0.14193438
epoch:  5 . Loss:  0.2593187093734741 . Tr Accuracy:  89.24704724409449 . Ts Accuracy:  80.71428571428571 Fr:  0.1414875
epoch:  6 . Loss:  0.2453763782978058 . Tr Accuracy:  90.87106299212599 . Ts Accuracy:  82.27678571428571 Fr:  0.14467968
epoch:  7 . Loss:  0.26052993535995483 . Tr Accuracy:  91.97834645669292 . Ts Accuracy:  79.6875 Fr:  0.13881172
epoch:  8 . Loss:  0.13671623170375824 . Tr Accuracy:  93.17175196850394 . Ts Accuracy:  82.58928571428571 Fr:  0.13888046
epoch:  9 . Loss:  0.20897